In [1]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print(type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


In [3]:
def Plural(palavra):
    """Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
        
    Returns:
        str: A string with 's' added to it.
    """
    return '{}s'.format(palavra)

print(Plural('gato'))

gatos


In [4]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
        
    Returns:
        str: A string with 's' added to it.



In [5]:
assert Plural('rato') == 'ratos', 'resultado incorreto!'
print('OK')

OK


In [8]:
pluralRDD = palavrasRDD.map(Plural)
print(pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [10]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'],'valores incorretos!'
print('OK')

OK


In [11]:
pluralLambdaRDD = palavrasRDD.map(lambda x:'{}s'.format(x))
print(pluralLambdaRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [12]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'],'valores incorretos!'
print('OK')

OK


In [13]:
pluralTamanho = (pluralRDD.map(lambda x:len(x)).collect())
print(pluralTamanho)

[5, 9, 5, 5, 5]


In [14]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print ("OK")

OK


In [15]:
palavraPar = palavrasRDD.map(lambda x:(x,1))
print(palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]
